# Sudoku Machine Learning Model

In this notebook we will train a machine learning model to solve sudokus

## Data preparation

The first step is to read and prepare the provided data. The data used for training the model is publicly available on Kaggle under the following URL:

https://www.kaggle.com/datasets/bryanpark/sudoku

The puzzles are reshaped to 9x9 array since this will be the input format for our model. The solutions will be reshaped to a 81x1 array for macthing the output format of our model.
We will also apply normalization on our data to improve the model performance.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

def prepare_data(file): 
    data = pd.read_csv(file)

    feat_raw = data['quizzes']
    label_raw = data['solutions']

    feat = []
    label = []

    # Reshape puzzles to 9x9 array
    for i in feat_raw:
        x = np.array([int(j) for j in i]).reshape((9,9,1))
        feat.append(x)
    
    # Normalize values
    feat = np.array(feat)
    feat = feat/9
    feat -= .5    
    
    # Reshape solutions to 81x1 array
    for i in label_raw:
        x = np.array([int(j) for j in i]).reshape((81,1)) - 1
        label.append(x)   
    
    label = np.array(label)
    
    # Remove raw data from memory
    del(feat_raw)
    del(label_raw)    

    # Split to 80% training and 20% validation
    x_train, x_test, y_train, y_test = train_test_split(feat, label, test_size=0.20)
    
    return x_train, x_test, y_train, y_test


## Model creation

In this step, we will create our model. By its nature, a Sudoku has some semantic meaning included and therefore we will use an approach based on Convolutional Neural Network (CNN) with Tensorflow (keras).

In [ ]:
import keras
from keras.layers import Activation
from keras.layers import Conv2D, BatchNormalization, Dense, Flatten, Reshape

def get_model():
    model = keras.models.Sequential()

    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

    model.add(Flatten())
    model.add(Dense(81*9))
    model.add(Reshape((-1, 9)))
    model.add(Activation('softmax'))
    
    return model
 

## Load the data

We load the data and split to training and validation datasets.

The dataset can be downloaded here:
https://www.kaggle.com/datasets/bryanpark/sudoku

In [3]:
x_train, x_test, y_train, y_test = prepare_data('sudoku.csv')

## Train the model

In this chapter we will finally train our model. We will introduce a callback function that would reduce the learning rate in case of an unchanging accuracy on the validation set (val_accuracy).

In [4]:
from keras.callbacks import Callback, ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=3,
    verbose=1,
    min_lr=1e-6
)
callbacks_list = [reduce_lr]

We will used the Adam optimizer with an initial learning_rate of 0.001.
The model is then initialized and trained with data. From performing several tests I could not notice a severe difference training for more than five epochs.

After the training, we save the model so that we can access it later from our web application.

In [7]:
model = get_model()

adam = keras.optimizers.Adam(learning_rate=.001)
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=adam, 
              metrics=['accuracy'])

model.fit(x_train, y_train, 
          validation_data=(x_test,y_test), batch_size=320, 
          epochs=5, verbose=1, callbacks=callbacks_list)

model.save("sudoku.h5")

Epoch 1/5
2500/2500 [==============================] - 275s 110ms/step - loss: 0.6192 - accuracy: 0.7422 - val_loss: 0.3969 - val_accuracy: 0.8157 - lr: 0.0010
Epoch 2/5
2500/2500 [==============================] - 272s 109ms/step - loss: 0.3795 - accuracy: 0.8219 - val_loss: 0.3718 - val_accuracy: 0.8240 - lr: 0.0010
Epoch 3/5
2500/2500 [==============================] - 274s 110ms/step - loss: 0.3593 - accuracy: 0.8309 - val_loss: 0.3633 - val_accuracy: 0.8278 - lr: 0.0010
Epoch 4/5
2500/2500 [==============================] - 273s 109ms/step - loss: 0.3460 - accuracy: 0.8384 - val_loss: 0.3599 - val_accuracy: 0.8298 - lr: 0.0010
Epoch 5/5
2500/2500 [==============================] - 273s 109ms/step - loss: 0.3353 - accuracy: 0.8449 - val_loss: 0.3589 - val_accuracy: 0.8303 - lr: 0.0010
